In [13]:
# to import .py module
import sys
import os
ROOT_DIR = os.path.abspath(os.curdir)[:-11]
sys.path.append(os.path.abspath(ROOT_DIR))
# from __future__ import absolute_import

In [14]:
import codecs
import json
import Preprocess
from Parameters import *
import numpy as np
import keras
from keras.models import Sequential, Model, load_model, save_model
from keras.layers import Embedding, Input, Dense, Reshape, Concatenate, Flatten, Bidirectional, LSTM, Activation, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from time import time
import multiprocessing as mp
import sys
import math
import tensorflow as tf
import timeit
import CRF

### Bert model

In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import json

def get_model(model_url, max_seq_length):
  labse_layer = hub.KerasLayer(model_url, trainable=True)

  # Define input.
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                         name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                     name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="segment_ids")

  # LaBSE layer.
  pooled_output,  _ = labse_layer([input_word_ids, input_mask, segment_ids])

  # The embedding is l2 normalized.
  pooled_output = tf.keras.layers.Lambda(
      lambda x: tf.nn.l2_normalize(x, axis=1))(pooled_output)

  # Define model.
  return tf.keras.Model(
        inputs=[input_word_ids, input_mask, segment_ids],
        outputs=pooled_output), labse_layer

max_seq_length = 64
labse_model, labse_layer = get_model(
    model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=max_seq_length)

In [11]:
import bert

vocab_file = labse_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = labse_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def create_input(input_strings, tokenizer, max_seq_length):

  input_ids_all, input_mask_all, segment_ids_all = [], [], []
  for input_string in input_strings:
    # Tokenize input.
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)

    # Padding or truncation.
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    segment_ids_all.append([0] * max_seq_length)

  return np.array(input_ids_all), np.array(input_mask_all), np.array(segment_ids_all)

def encode(input_text):
  input_ids, input_mask, segment_ids = create_input(
    input_text, tokenizer, max_seq_length)
  return labse_model([input_ids, input_mask, segment_ids])

# Model

## Load data and define necessary parameters, functions

In [3]:
import json
with open(TRAIN_DATA_FILE, encoding='utf8') as f:
  trainset = json.load(f)

In [4]:
with open(file=NORM_ADDS_FILE, mode='r', encoding='utf-8') as f:
  NORM_ADDS = json.load(fp=f)

with open(file=ID2id_FILE, mode='r', encoding='utf-8') as f:
  ID2id = json.load(fp=f)

with open(file=id2ID_FILE, mode='r', encoding='utf-8') as f:
  id2ID = json.load(fp=f)

with open(file=id2norm_add_FILE, mode='r', encoding='utf-8') as f:
  id2norm_add= json.load(fp=f)

In [5]:
norm_embeddings = np.load(NORM_EMBEDDING_FILE, allow_pickle=True)
NT_norm_embeddings = np.load(NT_NORM_EMBEDDING_FILE, allow_pickle=True)

In [ ]:
# noisy_add = trainset['data'][119]['noisy_add']

In [ ]:
# entities = CRF.detect_entity(noisy_add)
# entities

In [6]:
entities2index = {'street': 0, 'ward': 1, 'district': 2, 'city': 3}

In [7]:
def create_type_add_vector(noisy_add):
  entities = CRF.detect_entity(noisy_add)
  type_add_vector = np.zeros((1,4))
  for entity in entities:
    if entity == 'name':
      pass
    else:
      index = entities2index[entity]
      type_add_vector[0, index] = 1
  return type_add_vector

In [8]:
# for a sample in trainset, get id of norm_add coresponding to noisy_add of this sample
def get_norm_id(sample):
  return list(sample['std_add'].keys())[0]

In [9]:
# norm_embeddings.shape
dim = 772
num_of_norm = 34481
num_negative = 1023
SNN_layers = [512, 256, 128]
MLP_layers = [64, 32, 16, 8]

## Generator

In [10]:
def concat(v,type_add_vector):
  return np.concatenate((v, type_add_vector), axis=1)

In [11]:
def generator(trainset, num_negative, id2norm_add, dim):
  # for i, sample in enumerate(trainset['data'][:100]):
  for i, sample in enumerate(trainset['data'][:32932]):
    batch_size = num_negative + 1
    batch_indexes = np.array([i for i in range(batch_size)])
    noisy_adds, norm_adds, labels = [], [], []
    # noisy add có thể hoa hoặc thường, đưa về chữ thường hết và loại bỏ đi punctuation
    noisy_add = preprocess.remove_punctuation(CRF.get_better_add(sample['noisy_add']).lower())

    id = get_norm_id(sample)
    ID_ = int(id2ID[id])

    type_add_vector = create_type_add_vector(sample['noisy_add'])
    noisy_add_vector = concat(np.array(encode([noisy_add])), type_add_vector)

    # kiểm tra xem noisy_add truyền vào là thường hay hoa
    if noisy_add != preprocess.remove_tone_of_text(noisy_add):
      noisy_adds.append(noisy_add_vector)
      norm_adds.append(norm_embeddings[ID_, :].reshape(1, -1))
      labels.append(1)

      # nagative sample
      for t in range(num_negative):
        j = str(np.random.randint(num_of_norm))
        while (j not in id2norm_add) or (j == id):
          j = str(np.random.randint(num_of_norm))

        ID = int(id2ID[j])
        noisy_adds.append(noisy_add_vector)
        norm_adds.append(norm_embeddings[ID, :].reshape(1, -1))
        labels.append(0)

    else:
      noisy_adds.append(noisy_add_vector)
      norm_adds.append(NT_norm_embeddings[ID_, :].reshape(1, -1))
      labels.append(1)

      # negative sample
      for t in range(num_negative):
        j = str(np.random.randint(num_of_norm))
        while (j not in id2norm_add) or (j == id):
          j = str(np.random.randint(num_of_norm))

        ID = int(id2ID[j])
        noisy_adds.append(noisy_add_vector)
        norm_adds.append(NT_norm_embeddings[ID, :].reshape(1, -1))
        labels.append(0)

    np.random.shuffle(batch_indexes)
    yield [np.array(noisy_adds).reshape(batch_size, dim)[batch_indexes], np.array(norm_adds).reshape(batch_size, dim)[batch_indexes]], np.array(labels)[batch_indexes]

## Model

In [13]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, input_dim):
        super(CustomLayer, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim,), dtype="float32"),
            trainable=True,
        )
    def call(self, input_):
        return tf.multiply(input_, self.w)

def get_SNN_model(dim,
                  SNN_layers,
                  MLP_layers):
    num_SNN_layers = len(SNN_layers)

    noisy_input         = Input(shape=(dim,), dtype=np.float32, name='noisy_input')
    norm_input          = Input(shape=(dim,), dtype=np.float32, name='norm_input')

    noisy_latent = noisy_input
    norm_latent = norm_input
    # SNN layer
    for idx in range(num_SNN_layers):
      Dense_layer = Dense(units=SNN_layers[idx]) 
      noisy_latent = Dense_layer(noisy_latent)
      norm_latent = Dense_layer(norm_latent)

      Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
      noisy_latent = Activation_layer(noisy_latent)
      norm_latent = Activation_layer(norm_latent)

    L1_layer = keras.layers.Lambda(lambda tensors:keras.backend.abs(tensors[0] - tensors[1]))
    predict_vector = L1_layer([noisy_latent, norm_latent])

    # MLP layers
    for idx in range(len(MLP_layers)):
      Dense_layer = Dense(units=MLP_layers[idx]) 
      predict_vector = Dense_layer(predict_vector)

      Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
      predict_vector = Activation_layer(predict_vector)

    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(predict_vector)
    
    # Connect the inputs with the outputs
    siamese_net = keras.models.Model(inputs=[noisy_input,norm_input],outputs=prediction)
    
    # return the model
    return siamese_net
keras.backend.clear_session()

In [14]:
SNN_model = get_SNN_model(dim, SNN_layers, MLP_layers)
SNN_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
noisy_input (InputLayer)        [(None, 772)]        0                                            
__________________________________________________________________________________________________
norm_input (InputLayer)         [(None, 772)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          395776      noisy_input[0][0]                
                                                                 norm_input[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 512)          0           dense[0][0]           

In [15]:
tf.keras.utils.plot_model(SNN_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
SNN_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.binary_crossentropy)

# Train

In [ ]:
losses = []
start = timeit.default_timer()
for i in range(20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1301s 39ms/step - loss: 0.0213
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_1_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_1_epoches.snn/assets


32932/32932 [==============================] - 1299s 39ms/step - loss: 9.0034e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_2_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_2_epoches.snn/assets


32932/32932 [==============================] - 1292s 39ms/step - loss: 6.9460e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_3_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_3_epoches.snn/assets


   4720/Unknown - 194s 41ms/step - loss: 6.2243e-04

In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_3_epoches.snn')
start = timeit.default_timer()
for i in range(3, 20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 2055s 62ms/step - loss: 5.8007e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_4_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_4_epoches.snn/assets


32932/32932 [==============================] - 2018s 61ms/step - loss: 4.9296e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_5_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_5_epoches.snn/assets


32932/32932 [==============================] - 1994s 61ms/step - loss: 4.3467e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_6_epoches.snn/assets


...

32932/32932 [==============================] - 2001s 61ms/step - loss: 2.4712e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_12_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_12_epoches.snn/assets


32932/32932 [==============================] - 2003s 61ms/step - loss: 2.2622e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_13_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_13_epoches.snn/assets


   7303/Unknown - 444s 61ms/step - loss: 2.2560e-04

In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_13_epoches.snn')
start = timeit.default_timer()
for i in range(13, 35):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1300s 39ms/step - loss: 2.1273e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_14_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_14_epoches.snn/assets


32932/32932 [==============================] - 1265s 38ms/step - loss: 2.0479e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_15_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_15_epoches.snn/assets


32932/32932 [==============================] - 1293s 39ms/step - loss: 1.9486e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_16_epoches.snn/assets


...

32932/32932 [==============================] - 1224s 37ms/step - loss: 1.0556e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_34_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_34_epoches.snn/assets


32932/32932 [==============================] - 1233s 37ms/step - loss: 1.0650e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_35_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_35_epoches.snn/assets


Time:  28179.035810036
[0.00021273257152643055, 0.00020479364320635796, 0.00019486290693748742, 0.00018231708963867277, 0.0001757233403623104, 0.00016871487605385482, 0.00016136823978740722, 0.00015405939484480768, 0.0001482893858337775, 0.00014619415742345154, 0.00013502234651241452, 0.0001365508505841717, 0.00013006414519622922, 0.00012558359594549984, 0.000126219485537149, 0.00012108778901165351, 0.00012369292380753905, 0.00011711976549122483, 0.00011166442709509283, 0.00010740996367530897, 0.00010556317283771932, 0.00010649763862602413]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_35_epoches.snn')
start = timeit.default_timer()
for i in range(35, 50):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1327s 40ms/step - loss: 1.0303e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_36_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_36_epoches.snn/assets


32932/32932 [==============================] - 1285s 39ms/step - loss: 1.0077e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_37_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_37_epoches.snn/assets


32932/32932 [==============================] - 1310s 40ms/step - loss: 9.5319e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_38_epoches.snn/assets


...

32932/32932 [==============================] - 1242s 38ms/step - loss: 8.1474e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_49_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_49_epoches.snn/assets


32932/32932 [==============================] - 1292s 39ms/step - loss: 7.9526e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_50_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_50_epoches.snn/assets


Time:  18904.014745855
[0.00010303225280949846, 0.00010077057231683284, 9.531855175737292e-05, 9.330350439995527e-05, 9.494724508840591e-05, 9.354338544653729e-05, 9.001012949738652e-05, 9.58739037741907e-05, 8.69752693688497e-05, 8.483693090965971e-05, 8.446424908470362e-05, 8.331658318638802e-05, 8.230508683482185e-05, 8.147393964463845e-05, 7.952564919833094e-05]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_50_epoches.snn')
start = timeit.default_timer()
for i in range(50, 75):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1266s 38ms/step - loss: 7.4792e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_51_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_51_epoches.snn/assets


32932/32932 [==============================] - 1225s 37ms/step - loss: 7.8885e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_52_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_52_epoches.snn/assets


32932/32932 [==============================] - 1220s 37ms/step - loss: 7.6788e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_53_epoches.snn/assets


...

32932/32932 [==============================] - 1240s 38ms/step - loss: 5.8330e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_74_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_74_epoches.snn/assets


32932/32932 [==============================] - 1245s 38ms/step - loss: 5.6023e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_75_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_75_epoches.snn/assets


Time:  30926.374890240997
[7.479197665816173e-05, 7.888455002103001e-05, 7.67875972087495e-05, 7.381986506516114e-05, 7.318369898712263e-05, 7.35923313186504e-05, 7.594294584123418e-05, 7.118812936823815e-05, 7.050282874843106e-05, 7.180124521255493e-05, 6.988534732954577e-05, 7.121422095224261e-05, 6.492677493952215e-05, 6.478945579146966e-05, 6.650112482020631e-05, 6.198595656314865e-05, 6.615419988520443e-05, 6.637966725975275e-05, 6.17397454334423e-05, 6.535967258969322e-05, 6.10142778896261e-05, 6.212886364664882e-05, 6.095664139138535e-05, 5.833011891809292e-05, 5.602336023002863e-05]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_75_epoches.snn')
start = timeit.default_timer()
for i in range(75, 100):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1328s 40ms/step - loss: 5.8441e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_76_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_76_epoches.snn/assets


32932/32932 [==============================] - 1291s 39ms/step - loss: 5.4737e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_77_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_77_epoches.snn/assets


32932/32932 [==============================] - 1257s 38ms/step - loss: 5.8109e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_78_epoches.snn/assets


...

32932/32932 [==============================] - 1300s 39ms/step - loss: 4.7763e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_99_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_99_epoches.snn/assets


32932/32932 [==============================] - 1295s 39ms/step - loss: 4.5120e-05
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_100_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/AD_model_1.4.4.2/SNN_100_epoches.snn/assets


Time:  31997.37664702
[5.844053885084577e-05, 5.473746932693757e-05, 5.8108540542889386e-05, 5.70437332498841e-05, 6.0499176470329985e-05, 5.5390835768776014e-05, 5.537509423447773e-05, 5.480108302435838e-05, 5.37642408744432e-05, 5.672886618413031e-05, 5.178036735742353e-05, 5.12489968969021e-05, 5.498579776030965e-05, 5.2396964747458696e-05, 5.262072227196768e-05, 5.091861021355726e-05, 4.870855264016427e-05, 5.2029401558684185e-05, 4.955232725478709e-05, 5.224877895670943e-05, 4.9191668949788436e-05, 4.94468622491695e-05, 4.711630754172802e-05, 4.776317291543819e-05, 4.5119813876226544e-05]
